<a href="https://colab.research.google.com/github/Visors/d2l-zh-2/blob/main/tensorflow/chapter_deep-learning-computation/read-write.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 读写文件

到目前为止，我们讨论了如何处理数据，
以及如何构建、训练和测试深度学习模型。
然而，有时我们希望保存训练的模型，
以备将来在各种环境中使用（比如在部署中进行预测）。
此外，当运行一个耗时较长的训练过程时，
最佳的做法是定期保存中间结果，
以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。
因此，现在是时候学习如何加载和存储权重向量和整个模型了。

## (**加载和保存张量**)

对于单个张量，我们可以直接调用`load`和`save`函数分别读写它们。
这两个函数都要求我们提供一个名称，`save`要求将要保存的变量作为输入。


In [1]:
import numpy as np
import tensorflow as tf

x = tf.range(4)
np.save('x-file.npy', x)

我们现在可以将存储在文件中的数据读回内存。


In [2]:
x2 = np.load('x-file.npy', allow_pickle=True)
x2

array([0, 1, 2, 3], dtype=int32)

我们可以[**存储一个张量列表，然后把它们读回内存。**]


In [3]:
y = tf.zeros(4)
np.save('xy-files.npy', [x, y])
x2, y2 = np.load('xy-files.npy', allow_pickle=True)
(x2, y2)

(array([0., 1., 2., 3.]), array([0., 0., 0., 0.]))

我们甚至可以(**写入或读取从字符串映射到张量的字典**)。
当我们要读取或写入模型中的所有权重时，这很方便。


In [4]:
mydict = {'x': x, 'y': y}
np.save('mydict.npy', mydict)
mydict2 = np.load('mydict.npy', allow_pickle=True)
'''
mydict2 = np.load('mydict.npy')
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-5-2345116e4894> in <cell line: 0>()
      2 np.save('mydict.npy', mydict)
      3 # mydict2 = np.load('mydict.npy', allow_pickle=True)
----> 4 mydict2 = np.load('mydict.npy')
      5 mydict2

1 frames
/usr/local/lib/python3.11/dist-packages/numpy/lib/format.py in read_array(fp, allow_pickle, pickle_kwargs, max_header_size)
    793         # The array contained Python objects. We need to unpickle the data.
    794         if not allow_pickle:
--> 795             raise ValueError("Object arrays cannot be loaded when "
    796                              "allow_pickle=False")
    797         if pickle_kwargs is None:

ValueError: Object arrays cannot be loaded when allow_pickle=False
'''
mydict2

array({'x': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3], dtype=int32)>, 'y': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>},
      dtype=object)

## [**加载和保存模型参数**]

保存单个权重向量（或其他张量）确实有用，
但是如果我们想保存整个模型，并在以后加载它们，
单独保存每个向量则会变得很麻烦。
毕竟，我们可能有数百个参数散布在各处。
因此，深度学习框架提供了内置函数来保存和加载整个网络。
需要注意的一个重要细节是，这将保存模型的参数而不是保存整个模型。
例如，如果我们有一个3层多层感知机，我们需要单独指定架构。
因为模型本身可以包含任意代码，所以模型本身难以序列化。
因此，为了恢复模型，我们需要用代码生成架构，
然后从磁盘加载参数。
让我们从熟悉的多层感知机开始尝试一下。


In [24]:
# @tf.keras.utils.register_keras_serializable()
# class MLP(tf.keras.Model):
#     def __init__(self):
#         super().__init__()
#         self.flatten = tf.keras.layers.Flatten()
#         self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
#         self.out = tf.keras.layers.Dense(units=10)

#     # def build(self, input_shape):
#     #     super().build(input_shape)

#     def call(self, inputs):
#         x = self.flatten(inputs)
#         x = self.hidden(x)
#         return self.out(x)
@tf.keras.utils.register_keras_serializable()
class MLP(tf.keras.Model):
    def __init__(self, **kwargs):  # 接受父类参数
        super().__init__(**kwargs)  # 传递父类参数
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256, activation="relu")
        self.out = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        return self.out(x)

    def get_config(self):
        # 包含父类的配置（如 trainable, dtype 等）
        config = super().get_config()
        return config

    @classmethod
    def from_config(cls, config):
        # 显式处理父类的配置
        return cls(**config)

net = MLP()
X = tf.random.uniform((2, 20))
Y = net(X)

接下来，我们[**将模型的参数存储在一个叫做“mlp.params”的文件中。**]


In [25]:
# net.save_weights('mlp.params')
net.save_weights('mlp.weights.h5')
print(net.weights)

[<Variable path=mlp_6/dense_12/kernel, shape=(20, 256), dtype=float32, value=[[-0.07513162  0.05723687  0.05808729 ... -0.07338562 -0.0593318
   0.12891558]
 [ 0.03308995  0.06979074  0.07231967 ...  0.09212859  0.05531348
  -0.0923776 ]
 [-0.14568037 -0.12573588 -0.00111495 ... -0.05840454  0.02016944
  -0.00835845]
 ...
 [ 0.12687671  0.05443908 -0.11200537 ...  0.08015789  0.07001048
   0.12830433]
 [-0.07906099  0.00335337 -0.02530362 ... -0.00351118 -0.09068489
  -0.0826921 ]
 [ 0.14097118 -0.13957466  0.09202006 ... -0.07460159  0.00472659
  -0.02970877]]>, <Variable path=mlp_6/dense_12/bias, shape=(256,), dtype=float32, value=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

为了恢复模型，我们[**实例化了原始多层感知机模型的一个备份。**]
这里我们不需要随机初始化模型参数，而是(**直接读取文件中存储的参数。**)


In [26]:
clone = MLP()
# clone.load_weights('mlp.params')
X_clone = tf.random.uniform((2, 20))
clone(X_clone)
clone.load_weights('mlp.weights.h5')
print(clone.weights)

[<Variable path=mlp_7/dense_14/kernel, shape=(20, 256), dtype=float32, value=[[-0.07513162  0.05723687  0.05808729 ... -0.07338562 -0.0593318
   0.12891558]
 [ 0.03308995  0.06979074  0.07231967 ...  0.09212859  0.05531348
  -0.0923776 ]
 [-0.14568037 -0.12573588 -0.00111495 ... -0.05840454  0.02016944
  -0.00835845]
 ...
 [ 0.12687671  0.05443908 -0.11200537 ...  0.08015789  0.07001048
   0.12830433]
 [-0.07906099  0.00335337 -0.02530362 ... -0.00351118 -0.09068489
  -0.0826921 ]
 [ 0.14097118 -0.13957466  0.09202006 ... -0.07460159  0.00472659
  -0.02970877]]>, <Variable path=mlp_7/dense_14/bias, shape=(256,), dtype=float32, value=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

由于两个实例具有相同的模型参数，在输入相同的`X`时，
两个实例的计算结果应该相同。
让我们来验证一下。


In [27]:
Y_clone = clone(X)
Y_clone == Y

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

## 小结

* `save`和`load`函数可用于张量对象的文件读写。
* 我们可以通过参数字典保存和加载网络的全部参数。
* 保存架构必须在代码中完成，而不是在参数中完成。

## 练习

1. 即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？
1. 假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？
1. 如何同时保存网络架构和参数？需要对架构加上什么限制？


练习 1

便于非代码工作者在更直观的可视化工具下分析模型。

练习 2

可以读取存储的模型参数，将前两层写成块，在新网络中合适的位置加入该块结构。需要注意提取出来层应当是完整的拷贝而不是原来层的引用。

In [30]:
# 练习 3
net.save('net.keras')
saved = tf.keras.models.load_model('net.keras')
saved.summary()
print(saved.weights)


# '''
# ---------------------------------------------------------------------------
# TypeError                                 Traceback (most recent call last)
# <ipython-input-11-eb1665006932> in <cell line: 0>()
#       1 # 练习 3
#       2 net.save('net.keras')
# ----> 3 saved = tf.keras.models.load_model('net.keras')
#       4 saved.summary()
#       5 print(saved.weights)

# 5 frames
# /usr/local/lib/python3.11/dist-packages/keras/src/saving/serialization_lib.py in _retrieve_class_or_fn(name, registered_name, module, obj_type, full_config, custom_objects)
#     801             return obj
#     802
# --> 803     raise TypeError(
#     804         f"Could not locate {obj_type} '{name}'. "
#     805         "Make sure custom classes are decorated with "

# TypeError: Could not locate class 'MLP'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': None, 'class_name': 'MLP', 'config': {'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}}, 'registered_name': 'MLP', 'build_config': {'input_shape': [2, 20]}}
# '''

Model: "mlp_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_10 (Flatten)                 │ (2, 20)                     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (2, 256)                    │           5,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (2, 10)                     │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,946 (31.04 KB)

 Trainable params: 7,946 (31.04 KB)

 Non-trainable params: 0 (0.00 B)

[<Variable path=dense_20/kernel, shape=(20, 256), dtype=float32, value=[[-0.07513162  0.05723687  0.05808729 ... -0.07338562 -0.0593318
   0.12891558]
 [ 0.03308995  0.06979074  0.07231967 ...  0.09212859  0.05531348
  -0.0923776 ]
 [-0.14568037 -0.12573588 -0.00111495 ... -0.05840454  0.02016944
  -0.00835845]
 ...
 [ 0.12687671  0.05443908 -0.11200537 ...  0.08015789  0.07001048
   0.12830433]
 [-0.07906099  0.00335337 -0.02530362 ... -0.00351118 -0.09068489
  -0.0826921 ]
 [ 0.14097118 -0.13957466  0.09202006 ... -0.07460159  0.00472659
  -0.02970877]]>, <Variable path=dense_20/bias, shape=(256,), dtype=float32, value=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.

[Discussions](https://discuss.d2l.ai/t/1838)
